# 📈 Golden Cross & Death Cross Strategy Evaluation
**Author:** Kuo Liang  
**Date:** October 2025  

---

## 🧠 Introduction

This notebook analyzes the **Golden Cross** and **Death Cross** trading strategies using historical stock price data.  

These strategies rely on the relationship between **short-term** and **long-term moving averages** to identify potential market trend reversals:

- **Golden Cross** → when the short-term moving average (e.g., 50-day) crosses **above** the long-term moving average (e.g., 200-day).  
  It often signals a **bullish** trend.
- **Death Cross** → when the short-term moving average crosses **below** the long-term moving average.  
  It often signals a **bearish** trend.

In this notebook, we will:
1. 📊 Gather historical price data using Python (e.g., via `yfinance`).
2. 🔢 Calculate moving averages and identify crossover points.
3. 💰 Simulate trades based on the cross signals.
4. 📈 Evaluate performance against a simple buy-and-hold strategy.
5. 🎨 Visualize the results with price and signal charts.

---

### 🛠️ Libraries Used
- `pandas` – Data manipulation  
- `numpy` – Numerical computation  
- `matplotlib` / `plotly` – Visualization  
- `yfinance` – Stock price data retrieval  

---

### 🎯 Objective
To determine whether the Golden Cross and Death Cross strategies offer a statistically significant edge compared to a buy-and-hold approach.

---

*Let’s begin by importing the required libraries and fetching the data.*


In [ ]:
## Import modules & load class and functions

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [14]:
def fetch_data(tickers, start_date, end_date):
    """Fetch historical stock data from Yahoo Finance"""
    data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)
    return data


class MovingAverageCrossBacktest:
    def __init__(self, ticker, data, short_window=50, long_window=200, initial_capital=10000, verbose=False, showchart=False):
        """
        Initialize the backtesting system
        
        Parameters:
        - ticker: Stock symbol (e.g., 'AAPL', 'SPY')
        - short_window: Short-term moving average period (default: 50)
        - long_window: Long-term moving average period (default: 200)
        - initial_capital: Starting capital for backtesting
        - verbose: Print info in console (default:False)
        - showchart: Show chart when runing full backtest (default:False)
        """
        self.ticker = ticker
        self.short_window = short_window
        self.long_window = long_window
        self.initial_capital = initial_capital
        self.verbose = verbose
        self.showchart = showchart
        self.data = data
        self.signals = None
        self.portfolio = None
        
    def calculate_moving_averages(self):
        """Calculate short-term and long-term moving averages"""
        self.data['SMA_short'] = self.data['Close'].rolling(window=self.short_window).mean()
        self.data['SMA_long'] = self.data['Close'].rolling(window=self.long_window).mean()
        if self.verbose:
            print(f"Calculated {self.short_window}-day and {self.long_window}-day moving averages")
        
    def generate_signals(self):
        """
        Generate trading signals based on Golden Cross and Death Cross
        Golden Cross: Short MA crosses above Long MA (Buy signal)
        Death Cross: Short MA crosses below Long MA (Sell signal)
        """
        self.signals = pd.DataFrame(index=self.data.index)
        self.signals['price'] = self.data['Close']
        self.signals['SMA_short'] = self.data['SMA_short']
        self.signals['SMA_long'] = self.data['SMA_long']
        
        # Create signal: 1 when short MA > long MA, 0 otherwise
        self.signals['signal'] = 0.0
        self.signals['signal'] = np.where(self.signals['SMA_short'] > self.signals['SMA_long'], 1.0, 0.0)
        
        # Generate trading orders (difference in signal)
        self.signals['positions'] = self.signals['signal'].diff()
        
        # Identify Golden Cross (1.0) and Death Cross (-1.0)
        golden_crosses = self.signals[self.signals['positions'] == 1.0]
        death_crosses = self.signals[self.signals['positions'] == -1.0]
        if self.verbose:
            print(f"\nFound {len(golden_crosses)} Golden Cross events")
            print(f"Found {len(death_crosses)} Death Cross events")
        
        return self.signals
    
    def backtest_strategy(self):
        """Backtest the trading strategy and calculate returns"""
        # Initialize portfolio
        self.portfolio = pd.DataFrame(index=self.signals.index)
        self.portfolio['price'] = self.signals['price']
        self.portfolio['holdings'] = self.signals['signal'] * 100  # 100 shares when signal is active
        self.portfolio['cash'] = self.initial_capital
        self.portfolio['total'] = 0.0
        self.portfolio['returns'] = 0.0
        
        # More realistic backtesting with position tracking
        position = 0  # Number of shares held
        cash = self.initial_capital
        portfolio_value = []
        
        for i in range(len(self.signals)):
            price = self.signals['price'].iloc[i]
            signal = self.signals['positions'].iloc[i]
            
            # Golden Cross - Buy signal
            if signal == 1.0 and cash > 0:
                shares_to_buy = int(cash / price)
                position += shares_to_buy
                cash -= shares_to_buy * price
                
            # Death Cross - Sell signal
            elif signal == -1.0 and position > 0:
                cash += position * price
                position = 0
            
            # Calculate total portfolio value
            total_value = cash + (position * price)
            portfolio_value.append(total_value)
        
        self.portfolio['total'] = portfolio_value
        self.portfolio['returns'] = self.portfolio['total'].pct_change()
        
        # Calculate buy and hold strategy for comparison
        buy_hold_shares = self.initial_capital / self.signals['price'].iloc[self.long_window]
        self.portfolio['buy_hold'] = buy_hold_shares * self.signals['price']
        
        # Calculate performance metrics
        final_value = self.portfolio['total'].iloc[-1]
        total_return = ((final_value - self.initial_capital) / self.initial_capital) * 100
        
        buy_hold_final = self.portfolio['buy_hold'].iloc[-1]
        buy_hold_return = ((buy_hold_final - self.initial_capital) / self.initial_capital) * 100
        if self.verbose:
            print(f"\n{'='*60}")
            print(f"BACKTESTING RESULTS")
            print(f"{'='*60}")
            print(f"Initial Capital: ${self.initial_capital:,.2f}")
            print(f"\nGolden/Death Cross Strategy:")
            print(f"  Final Value: ${final_value:,.2f}")
            print(f"  Total Return: {total_return:.2f}%")
            print(f"\nBuy & Hold Strategy:")
            print(f"  Final Value: ${buy_hold_final:,.2f}")
            print(f"  Total Return: {buy_hold_return:.2f}%")
            print(f"\nStrategy vs Buy & Hold: {total_return - buy_hold_return:+.2f}%")
            print(f"{'='*60}")
        
        return self.portfolio
    
    def plot_results(self):
        """Visualize the backtesting results"""
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
        
        # Plot 1: Price, Moving Averages, and Signals
        ax1.plot(self.data.index, self.data['Close'], label='Close Price', alpha=0.7, linewidth=1.5)
        ax1.plot(self.signals.index, self.signals['SMA_short'], 
                label=f'{self.short_window}-day SMA', alpha=0.8, linewidth=1.5)
        ax1.plot(self.signals.index, self.signals['SMA_long'], 
                label=f'{self.long_window}-day SMA', alpha=0.8, linewidth=1.5)
        
        # Plot Golden Cross (Buy signals)
        golden_cross = self.signals[self.signals['positions'] == 1.0]
        ax1.scatter(golden_cross.index, golden_cross['price'], 
                   color='green', marker='^', s=150, label='Golden Cross (Buy)', zorder=5)
        
        # Plot Death Cross (Sell signals)
        death_cross = self.signals[self.signals['positions'] == -1.0]
        ax1.scatter(death_cross.index, death_cross['price'], 
                   color='red', marker='v', s=150, label='Death Cross (Sell)', zorder=5)
        
        ax1.set_ylabel('Price ($)', fontsize=12)
        ax1.set_title(f'{self.ticker} - Golden Cross & Death Cross Strategy', fontsize=14, fontweight='bold')
        ax1.legend(loc='best')
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Portfolio Value
        ax2.plot(self.portfolio.index, self.portfolio['total'], 
                label='Strategy Portfolio Value', linewidth=2, color='blue')
        ax2.plot(self.portfolio.index, self.portfolio['buy_hold'], 
                label='Buy & Hold Portfolio Value', linewidth=2, color='orange', linestyle='--')
        ax2.axhline(y=self.initial_capital, color='gray', linestyle=':', 
                   label='Initial Capital', linewidth=1.5)
        
        ax2.set_xlabel('Date', fontsize=12)
        ax2.set_ylabel('Portfolio Value ($)', fontsize=12)
        ax2.set_title('Portfolio Performance Comparison', fontsize=14, fontweight='bold')
        ax2.legend(loc='best')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
    def run_full_backtest(self):
        """Run the complete backtesting pipeline"""
        self.calculate_moving_averages()
        self.generate_signals()
        self.backtest_strategy()
        if self.showchart:
            self.plot_results()

## Run backtest

In [10]:
# Example usage
if __name__ == "__main__":
    # Set parameters
    TICKER = 'AAPL'  # Change to any stock ticker
    START_DATE = '2020-01-01'
    END_DATE = '2024-12-31'
    INITIAL_CAPITAL = 10000
    
    # Create backtester instance
    backtester = MovingAverageCrossBacktest(
        ticker=TICKER,
        start_date=START_DATE,
        end_date=END_DATE,
        short_window=50,
        long_window=200,
        initial_capital=INITIAL_CAPITAL
    )
    
    # Run backtest
    backtester.run_full_backtest()
    
    # Access results
    print("\nFirst few signals:")
    print(backtester.signals[backtester.signals['positions'] != 0].head())

[*********************100%***********************]  1 of 1 completed


First few signals:
                 price   SMA_short    SMA_long  signal  positions
Date                                                             
2020-01-02   72.538506         NaN         NaN     0.0        NaN
2020-10-15  117.416748  113.729093   85.965763     1.0        1.0
2022-06-03  142.922256  156.255111  156.408178     0.0       -1.0
2022-09-26  148.427017  157.668041  157.631242     1.0        1.0
2022-10-07  137.912994  156.199682  156.349957     0.0       -1.0
